# Classification Algorithm using Pyspark

First install pyspark and the create and check if the session was created correctly

In [2]:
!pip install pyspark==2.4.5

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 217.8 MB 10 kB/s s eta 0:00:01    |███▎                            | 22.5 MB 13.3 MB/s eta 0:00:152     |█████████▍                      | 64.0 MB 43.7 MB/s eta 0:00:04     |██████████████████▊             | 127.2 MB 41.8 MB/s eta 0:00:03�██████████▎        | 158.6 MB 51.7 MB/s eta 0:00:02██████████████████████▍       | 166.3 MB 46.3 MB/s eta 0:00:02     |███████████████████████████▏    | 184.8 MB 46.3 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 40.8 MB/s eta 0:00:01
  Created whee

In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

Download the parquet data

In [5]:
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet

--2021-09-15 18:47:14--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/claimed/raw/master/coursera_ml/a2.parquet [following]
--2021-09-15 18:47:15--  https://github.com/IBM/claimed/raw/master/coursera_ml/a2.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/claimed/master/coursera_ml/a2.parquet [following]
--2021-09-15 18:47:15--  https://raw.githubusercontent.com/IBM/claimed/master/coursera_ml/a2.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting r

Now it’s time to have a look at the recorded sensor data. 

In [6]:
df=spark.read.load('a2.parquet')

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

Please create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import StringIndexer

stringIndexer = StringIndexer(inputCol="CLASS", outputCol="label")
vectorAssembler = VectorAssembler(inputCols = ["X","Y","Z"], outputCol = "features")
normalizer = Normalizer(inputCol = "features", outputCol = "features_norm", p = 1.0 )

Instantiate a classifier from the SparkML package and assign it to the classifier variable. 


In [15]:
from pyspark.ml.classification import GBTClassifier

classifier = GBTClassifier(labelCol = "label", featuresCol = "features_norm", maxIter = 10)



Let’s train and evaluate the model


In [16]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[stringIndexer, vectorAssembler ,normalizer ,classifier])

In [17]:
model = pipeline.fit(df)

In [18]:
prediction = model.transform(df)

In [19]:
prediction.show()

+-----+-----------+-------------------+-------------------+-------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|label|            features|       features_norm|       rawPrediction|         probability|prediction|
+-----+-----------+-------------------+-------------------+-------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|  0.0|[380.664340054951...|[0.49568998814276...|[0.32604262845078...|[0.65748021657963...|       0.0|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|  0.0|[104.743242992096...|[0.64607479930754...|[0.36769831142141...|[0.67598840532916...|       0.0|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|  0.0|[118.11469

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction) 

0.7909270216962525